# Order Book Dynamics

In this notebook, we explore how electronic markets actually match buyers and sellers. We'll use plain English and avoid heavy math.

## 1. Order Types

- **Limit Orders**
  - You say how much you want to trade *and* the exact price you’re willing to pay or accept.
  - Your order patiently waits in the electronic "book" until someone matches your price.
  - You can cancel it at any time if it doesn't fill.

- **Market Orders**
  - You tell the system, “Fill my trade right now at the best available price.”
  - This guarantees speed, not price—you might sweep through multiple price levels to get your size filled.
  - Market orders take liquidity; limit orders make liquidity.

## 2. The Bid–Ask Spread

- At any moment, the top **buy** price in the book is the **Bid**.
- The top **sell** price is the **Ask**.
- The **spread** is simply: Ask minus Bid.

- **Tight spreads** (small difference) mean low cost to trade immediately.
- **Wide spreads** mean higher implicit costs: if you buy at the ask and flip to sell at the bid, you lose that difference.

## 3. Picking a Simple "Price"

- **Midprice**
  - (Bid + Ask) / 2 provides a quick snapshot of where the market sits.

- **Microprice**
  - We weight the bid and ask by how many shares are sitting at each level.
  - If there’s a big pile of bids but only a few asks, the microprice shifts toward the ask, hinting at hidden buying pressure.

## 4. Why It Matters

- **Execution cost**: Market orders pay the spread; limit orders earn it.
- **Liquidity**: The depth of the book tells you how big an order you can place without moving the price too much.
- **Market signals**: Changes in the spread and book shape often signal upcoming volatility or large trades.

In [ ]:
# Load and prepare BNB/USDT 1-minute data (last 30 days)
import pandas as pd

path = '/Users/mchildress/Active Code/ts_basics/data/bnbusdt_1m.csv'
df = (
    pd.read_csv(path, parse_dates=['timestamp'])
      .set_index('timestamp')
      .sort_index()
      .iloc[-30*24*60:]   # keep only last 30 days
)

df.head()

In [ ]:
# Compute midprice and spread
df['midprice'] = (df['high'] + df['low']) / 2
df['spread']   = df['high'] - df['low']

import matplotlib.pyplot as plt

plt.figure(figsize=(12,4))
ax = plt.gca()
ax.plot(df.index, df['midprice'], label='Mid-price', color='C0')
ax.set_ylabel('Mid-price')

ax2 = ax.twinx()
ax2.plot(df.index, df['spread'], label='Spread', color='C1', alpha=0.6)
ax2.set_ylabel('Spread')

ax.set_title('BNB/USDT Mid-Price & Spread (1m, last 30 days)')
ax.legend(loc='upper left')
ax2.legend(loc='upper right')
plt.tight_layout()
plt.show()

## 5. Analysis

- **Mid-Price Trend**: Over the last 30 days, the mid-price for BNB/USDT has moved between about 620 and 780. We see clear periods of rising price (early spikes) followed by consolidations where the price bounces in a range.
- **Spread Behavior**: Most of the time, the spread stays low (under 5), indicating the market is liquid and you can trade quickly with very little cost. 
- **Spread Spikes**: Large spikes in the spread (sometimes above 20 or even 30) hint at moments of lower liquidity or volatility—perhaps around big news events or when order-book updates slowed down. During those spikes, placing a market order would be noticeably more expensive.
- **Takeaways**:
  - If you trade during low-spread periods, you pay only a tiny fee in the bid–ask difference.
  - If the spread jumps up, it’s a sign to pause or switch to limit orders, since market orders can cost a lot more.